# Residential Tree Planting project data prep

## Loading and formatting data

In [318]:
import pandas as pd
import os
import numpy as np
import geopy

In [319]:
#Rename excel files
old_names = list(os.listdir('./data'))

for i in old_names:
    if '.xlsx' not in i:
        old_names.remove(i)

old_names

['fall_2023.xlsx',
 'fall_2024.xlsx',
 'fall_2025.xlsx',
 'spring_2023.xlsx',
 'spring_2024_1.xlsx',
 'spring_2024_2.xlsx',
 'spring_2025.xlsx']

In [320]:
new_names = []
for i in old_names:
    new_name = i.lower().replace(" ","_").strip()
    new_name = new_name.replace("casey_", "").replace("tree_","").replace("trees_","").replace("planting_","")
    new_names.append(new_name)

new_names

['fall_2023.xlsx',
 'fall_2024.xlsx',
 'fall_2025.xlsx',
 'spring_2023.xlsx',
 'spring_2024_1.xlsx',
 'spring_2024_2.xlsx',
 'spring_2025.xlsx']

In [321]:
# ALREADY CHANGED NAMES THEREFORE NOT NECESSARY
# new_names[1] = 'spring_2023.xlsx'
# new_names[2] = 'spring_2024_2.xlsx'
# new_names[4] = 'fall_2024.xlsx'
# new_names[6] = 'spring_2024_1.xlsx'

# new_names

In [322]:
# ALREADY CHANGED NAMES THEREFORE NOT NECESSARY
# for i in old_names:
#     os.rename(i, new_names[old_names.index(i)])

In [323]:
# Read files into df
dfs = {}
for i in new_names:
    dfs[i] = pd.read_excel(("./data/"+i), header=0)

C:\Users\pmedina\AppData\Local\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [324]:
dfs.keys()

dict_keys(['fall_2023.xlsx', 'fall_2024.xlsx', 'fall_2025.xlsx', 'spring_2023.xlsx', 'spring_2024_1.xlsx', 'spring_2024_2.xlsx', 'spring_2025.xlsx'])

## Formatting data

In [325]:
# Variable name formatting
def clean_column_names(df):
    df.columns = (df.columns
               .str.strip()
               .str.lower()
               .str.replace(" ", "_")
               .str.replace("[()€$]", "",

                            regex=True))
    return df

In [326]:
for i in dfs:
    clean_column_names(dfs[i])

In [327]:
spring_2023 = dfs['spring_2023.xlsx']
fall_2023 = dfs['fall_2023.xlsx']
spring_2024_2 = dfs['spring_2024_2.xlsx']
fall_2025 = dfs['fall_2025.xlsx']
fall_2024 = dfs['fall_2024.xlsx']
spring_2025 = dfs['spring_2025.xlsx']
spring_2024_1 = dfs['spring_2024_1.xlsx']

In [328]:
def update_species(df):
    for index, row in df.iterrows():
        if ',' in row['species_1']:
            species = df.loc[index, 'species_1'].split(',')
            species_1 = species[0]
            species_2 = species[1]
            df.loc[index, 'species_1'] = species_1
            df.loc[index, 'species_2'] = species_2
        elif '2' in row['species_1']:
            species = row['species_1'].strip().replace('2','').replace('()','')
            df.loc[index, 'species_1'] = species
            df.loc[index, 'species_2'] = species
        else:
            row['species_2'] = ''
    return df

### Spring 2023

In [329]:
spring_2023.head(5)

,application_date;_planting_on_resid_property,area,district_#,first_and_last_name:,email_address:,phone_number:,address:,preferred_contact,consultation_date,casey_trees_date_planted,city_inspection_date,species_planted,notes_casey_trees
0,2022-05-14 21:50:25,Daniels Pk E,1,Kirstin Showalter,kirstinsd@gmail.com,7346492987,5121 Kenesaw St,Email,2023-02-28,NaN,NaN,"A.Hornbeam, Wil Oak",match
1,2022-11-11 13:10:39,Daniels Pk W,1,Anne Tolbert,ktolbertmd@gmail.com,301-613-3496,4908 Hollywood Rd,Email,2023-03-01,NaN,NaN,(2) Willow Oaks,no map
2,2023-02-14 11:03:46,Oak Sps W,1,Frances Sutphen,healingfls@verizon.net,13013950486,4822 Erie Street,Phone,2023-03-01,NaN,NaN,S. Magnolia,match
3,2023-02-24 08:53:47,Hollywood E,1,Donald Hays,hays.donald@gmail.com,270-320-2950,5017 Laguna Road,NaN,2023-02-27,NaN,NaN,S. Magnolia,match
4,2022-05-09 19:33:25,Autoville,1,Alan Hew,alanhew@gmail.com,3014047519,9118 Autoville Dr.,Email,2023-02-28,NaN,NaN,Am. Witch Hazel,match


In [330]:
# Drop unused columns and PII
spring_2023 = spring_2023.drop(['preferred_contact','application_date;_planting_on_resid_property','area','city_inspection_date','consultation_date','casey_trees_date_planted','notes_casey_trees', 'first_and_last_name:', 'email_address:','phone_number:'], axis = 1)

In [331]:
# Add new columns and rename remaining ones
spring_2023 = spring_2023.rename(columns={'species_planted':'species_1'})
spring_2023['species_2'] = ""
spring_2023['year'] = 2023
spring_2023 = updateSpecies(spring_2023)
spring_2023.head(5)

,district_#,address:,species_1,species_2,year
0,1,5121 Kenesaw St,A.Hornbeam,Wil Oak,2023
1,1,4908 Hollywood Rd,Willow Oaks,Willow Oaks,2023
2,1,4822 Erie Street,S. Magnolia,,2023
3,1,5017 Laguna Road,S. Magnolia,,2023
4,1,9118 Autoville Dr.,Am. Witch Hazel,,2023


In [332]:
spring_2023.head(5)

,district_#,address:,species_1,species_2,year
0,1,5121 Kenesaw St,A.Hornbeam,Wil Oak,2023
1,1,4908 Hollywood Rd,Willow Oaks,Willow Oaks,2023
2,1,4822 Erie Street,S. Magnolia,,2023
3,1,5017 Laguna Road,S. Magnolia,,2023
4,1,9118 Autoville Dr.,Am. Witch Hazel,,2023


### Fall 2023

In [333]:
fall_2023.head(5)

,casey_tree_planting_on_private_property_fall_2023,unnamed:_1,unnamed:_2,unnamed:_3,unnamed:_4,unnamed:_5,unnamed:_6,unnamed:_7,unnamed:_8,unnamed:_9,unnamed:_10,unnamed:_11,unnamed:_12
0,Application Date; Planting on Residential Prop...,Area,District #,First and Last Name:,Email Address:,Phone Number,Address:,Preferred Contact,Consultation date,Casey Trees date planted,City inspection date,Species planted,Hold Harmless Form completed
1,2022-06-25 00:00:00,CP Estates,3,Andrew Stephens,andrewstephens11@gmail.com,202-550-5392,7512 Wellesley Dr,email,2023-06-08 00:00:00,2024-10-30 00:00:00,2023-11-17 00:00:00,American Linden,X
2,2022-07-01 00:00:00,CP Woods,4,Bruce Simpson,ilovjazz@gmail.com,301-633-7736,3711 Marlbrough Way,email,2023-07-05 00:00:00,2023-11-03 00:00:00,2023-11-09 00:00:00,Red Maple,X
3,2023-03-09 00:00:00,Berwyn,2,Emily Friend,emmfriend@gmail.com,301-356-5396,4803 Ruatan St,email,2023-06-09 00:00:00,2023-11-03 00:00:00,2023-11-09 00:00:00,Tulip Poplar,X
4,2023-03-26 00:00:00,Hollywood W,1,Malcolm Williams,malcolmtwilliams@gmail.com,240-839-8554,4806 Lackawanna St,email,2023-06-26 00:00:00,2023-11-03 00:00:00,2023-11-09 00:00:00,Southern Magnolia,X


In [334]:
new_column_names = list(fall_2023.loc[0,])
fall_2023.columns = new_column_names
fall_2023 = clean_column_names(fall_2023)
fall_2023 = fall_2023.drop(['casey_trees_date_planted','hold_harmless_form_completed','preferred_contact','application_date;_planting_on_residential_property','area','city_inspection_date','consultation_date','casey_trees_date_planted','first_and_last_name:', 'email_address:','phone_number'], axis = 1)

In [335]:
fall_2023 = fall_2023.rename(columns={'species_planted':'species_1'})
fall_2023['year'] = 2023
fall_2023 = fall_2023.loc[1:] #Delete column names row
fall_2023 = update_species(fall_2023)
fall_2023.head(5)

,district_#,address:,species_1,year
1,3,7512 Wellesley Dr,American Linden,2023
2,4,3711 Marlbrough Way,Red Maple,2023
3,2,4803 Ruatan St,Tulip Poplar,2023
4,1,4806 Lackawanna St,Southern Magnolia,2023
5,1,9815 52nd Pl,Tulip Poplar,2023


### Spring 2024 - 1

In [351]:
spring_2024_1

,application_date;_planting_on_residential_property,city_district,first_and_last_name:,email_address:,phone_number,address:,consultation_date,casey_pl,city_insp,species_planted,hold_harmless_form_completed
0,2023-09-17,1,Anning Cheng,anningc@yahoo.com,757-268-8522,5109 Iroquois St,2023-12-08,2024-04-05,2024-04-23,Red Maple,X
1,2023-10-08,1,Kim Tolbert,ktolbertmd@gmail.com,301-613-3496,4908 Hollywood Rd,2023-12-08,2024-04-05,2024-04-23,"Beech, Red Maple",X
2,2023-10-14,1,Kristi Medley,corbettmedley@gmail.com,202-812-0014,4810 Lackawanna St,2023-11-27,2024-04-10,2024-04-23,Dogwood,X
3,2023-10-24,1,Mae Anderson-Chinnes,mae.chinnes@gmail.com,843-291-8153,5106 Lackawanna St,2023-12-08,2024-04-05,2024-04-24,"Southern Magnolia, Swamp White Oak",X
4,2023-11-17,1,Diana Sisson,dmpeloquin@gmail.com,540-226-1829,9735 53rd Ave,2023-11-27,2024-04-05,2024-04-19,"Red Maple, Dogwood",X
5,2023-10-28,1,Brittney Kuzila,brittneykuzila@gmail.com,202-805-2458,5012 Hollywood Rd,2023-12-12,2024-04-05,2024-04-23,Catalpa,X
6,2023-11-28,1,Shane Smith,srsmith716@gmail.com,412-916-6404,5014 Hollywood Rd,2023-12-14,2024-04-05,2024-04-23,"Amer Linden, Loblolly Pine",X
7,2023-12-15,1,Sanjay Agarwal,sanjayagarwal73@gmail.com,703-349-9464,9710 Rhode Is Ave,2023-12-19,2024-04-05,2024-04-23,Red Maple,X
8,2023-12-18,1,Jaime Theilen,jaimetheilen@gmail.com,502-295-3682,4904 Erie St,2023-01-12,2024-04-05,2024-04-19,2 River Birch,X
9,2023-12-27,1,Julio Hernandez,juliodeleon1987@gmail.com,301-312-3370,5001 Kenesaw St,2023-01-08,2024-04-05,2024-04-19,"Prince Oak, River Birch",X


In [353]:
spring_2024_1#.drop('application_date;_planting_on_residential_property', axis = 1)
del_columns = ['application', 'name','email','casey','form']

def column_clean(df, char_del):
    for column in list(df.columns):
        for i in char_del:
            if i in column:
                df.drop(column,axis = 1)


# list(spring_2024_1.columns)[0]
# if 
# spring_2024_1[list(spring_2024_1.columns)[0]]

In [355]:
spring_2024_1 = column_clean(spring_2024_1,del_columns)
spring_2024_1